In [1]:
import preprocessing
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import project_preprocessing_helper
import gc

In [2]:
df = preprocessing.read_application()

In [3]:
previous_app = preprocessing.read_previous_application()

In [4]:
bureau = preprocessing.read_bureau()

In [5]:
df = df.join(bureau, on='SK_ID_CURR', how='left')
del bureau
gc.collect()

0

In [6]:
df = df.join(previous_app, on='SK_ID_CURR', how='left')
del previous_app
gc.collect()

0

In [7]:
train, test, target, features_name = preprocessing.split_train_test_target(df)

In [8]:
from sklearn.preprocessing import StandardScaler, Imputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, ShuffleSplit
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.pipeline import Pipeline

In [18]:
pipe_ada = Pipeline([('imputer', Imputer(strategy = 'median')),
                    ('scaler', StandardScaler()),
                    ('clf', AdaBoostClassifier(n_estimators=100))
                    ])
pipe_rf = Pipeline([('imputer', Imputer(strategy = 'median')),
                    ('scaler', StandardScaler()),
                    ('clf', RandomForestClassifier(n_estimators=100, max_depth = 10, 
                                                   class_weight = 'balanced_subsample'))
                    ])

In [13]:
pipe_rf.fit(train, target)

Pipeline(memory=None,
         steps=[('imputer',
                 Imputer(axis=0, copy=True, missing_values='NaN',
                         strategy='median', verbose=0)),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('clf',
                 RandomForestClassifier(bootstrap=True,
                                        class_weight='balanced_subsample',
                                        criterion='gini', max_depth=10,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                     

In [22]:
submit = pipe_ada.predict_proba(test)[:,1]

In [26]:
submit = test[['SK_ID_CURR']]
submit['TARGET'] = pipe_ada.predict_proba(test)[:,1]
submit.to_csv('submit_app_previous_bureau_ada.csv', index=False)

##### submit

In [17]:
submit = test[['SK_ID_CURR']]
submit['TARGET'] = pipe_rf.predict_proba(test)[:,1]
submit.to_csv('submit_app_previous_bureau_rf.csv', index=False)

In [19]:
from sklearn.model_selection import cross_val_score

In [20]:
cross_val_score(pipe_ada, train, target, cv=3)

array([0.14809612, 0.9191122 , 0.91927726])

In [28]:
train.shape

(307500, 524)

In [35]:
train.corrwith(target)

AMT_ANNUITY                                                           -0.012814
AMT_CREDIT                                                            -0.030390
AMT_GOODS_PRICE                                                       -0.039671
AMT_INCOME_TOTAL                                                      -0.003970
AMT_REQ_CREDIT_BUREAU_DAY                                              0.002704
                                                                         ...   
previous_app_PRODUCT_COMBINATION_POS industry without interest_mean   -0.021916
previous_app_PRODUCT_COMBINATION_POS mobile with interest_mean         0.021582
previous_app_PRODUCT_COMBINATION_POS mobile without interest_mean     -0.002560
previous_app_PRODUCT_COMBINATION_POS other with interest_mean         -0.003912
previous_app_PRODUCT_COMBINATION_POS others without interest_mean     -0.002153
Length: 524, dtype: float64